# OpenSearch Hybrid 검색을 통한 RAG
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 이 노트북은 이전 노트북인 "02_KR_RAG_OpenSearch_Claude.ipynb" 이 완료 되었다고 가정 합니다.
    - 오픈서치 인텍스 관련 정보를 참조 합니다.

### Methods and Resources for Hybrid search with Re-ranking 
- Score Normalization
    - [MinMax based](https://towardsdatascience.com/text-search-vs-vector-search-better-together-3bd48eb6132a)
    - [z-socre based](https://towardsdatascience.com/hybrid-search-2-0-the-pursuit-of-better-search-ce44d6f20c08)
- Reciprocal Rank Fusion (RRF)
    - [Paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)
    - [Description](https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d)
- [LangChain API for Ensemble Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.



In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [2]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [3]:
from langchain.llms.bedrock import Bedrock

In [4]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={'max_tokens_to_sample': 512}
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f931c17b760>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512})

## Embedding 모델 선택

In [5]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [6]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 3. LangChain OpenSearch VectorStore 생성 
## 선수 조건
- 이전 노트북 02_1_KR_RAG_OpenSearch_Keyword.ipynb 또는 02_1_KR_RAG_OpenSearch_Semantic.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.

## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 OpenSearch ID/PW 를 입력을 해주세요.

In [7]:
os.environ["OpenSearch_UserName"] = "Type ID"
os.environ["OpenSearch_UserPassword"] = "Type PW"

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

In [8]:
opensearch_domain_endpoint = "<Type your domain endpoint>"
#opensearch_domain_endpoint = "https://search-genai-demo-domain-36etfsphifulgs7ckovsavchca.us-east-1.es.amazonaws.com"

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

index_name = "genai-demo-index-v1-with-tokenizer"

## LangChain OpenSearch VectorStore 생성

In [9]:
from langchain.vectorstores import OpenSearchVectorSearch

In [10]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Client 생성

In [11]:
from utils.opensearch import opensearch_utils

In [12]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 은전한잎 형태소 분석기 (seunjeon_tokenizer) 사용하기 in Opensearch
- 형태소 분석기에 대한 자세한 사항은 02_1_KR_RAG_OpenSearch_Keyword.ipynb 참고

### 인덱스 확인 (tokenization 확인)

In [13]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

{'genai-demo-index-v1-with-tokenizer': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                                'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                                  'type': 'keyword'}},
                                                                                                           'type': 'text'},
                                                                                                'timestamp': {'type': 'float'},
                                                                                                'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                         

# 4. 오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [14]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter

## (1) OpenSearch Vector 검색

### 프로프트 템플릿 생성

In [15]:
from utils.rag import run_RetrievalQA, show_context_used
from langchain.prompts import PromptTemplate

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [16]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### 필터 생성

In [17]:
filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

# query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
query = "타기관OTP 등록 방법 알려주세요"

boolean_filter = opensearch_utils.get_filter(
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ]
)

pprint(boolean_filter)

{'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}},
                     {'term': {'metadata.source': '신한은행'}}]}}


### LangChain RetrievalQA 를 통해 실행

In [18]:
result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=False,
    k=3
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(result['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(result['source_documents'])

##################################
query:  타기관OTP 등록 방법 알려주세요
boolean_filter:  {'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}}, {'term': {'metadata.source': '신한은행'}}]}}
##################################


### Answer ###
 인터넷뱅킹에서 타기관 OTP를 등록하려면 아래와 같은 방법이 있습니다:

1. 인터넷뱅킹 로그인 → 사용자관리 → 인터넷뱅킹관리 → OTP이용등록

2. 신한 쏠(SOL) 로그인 → 전체메뉴 → 설정/인증 → 이용중인 보안매체선택 → OTP이용등록

OTP 이용등록 후 재로그인을 해야 새로 등록한 보안매체가 적용됩니다.

자세한 내용은 신한은행 고객센터(1599-8000)로 문의하시기 바랍니다.


### Contexts ###
-----------------------------------------------
1. Chunk: 279 Characters
-----------------------------------------------
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다.
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록

 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 3, 'type': '인터넷뱅킹', 'timestamp': 1696755374.0453036}
----------------------------------------------

## (2) OpenSearch Keyword 검색
- OpenSearch 에 아래와 같은 Query 를 실행하여 결과 받는 것을 구현합니다.

![keyword_filter.png](img/keyword_filter.png)

In [19]:
from utils.opensearch import opensearch_utils
from utils.rag import get_keyword_similar_docs 
from langchain.chains.question_answering import load_qa_chain

In [20]:
filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
#query = "타기관OTP 등록 방법 알려주세요"

search_keyword_result = get_keyword_similar_docs(
    query=query,
    minimum_should_match=0,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    index_name=index_name,
    os_client=os_client,
    k=3,
    hybrid=False
)
print(search_keyword_result)

keyword search query: 
{'query': {'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}},
                               {'term': {'metadata.source': '신한은행'}}],
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '홈페이지 이용자아이디 여러 개 '
                                                          '사용할 수 있나요?'}}}]}},
 'size': 3}
[Document(page_content='ask: 즉시/예약 이체 서비스가 안되는 경우 어떻게 해야해?\nInformation: 이체서비스가 되지 않는 경우 여러가지 사유가 있을수 있으니, 신한은행 고객상담센터 1599-8000번으로 문의 주시면 자세히 상담해드리겠습니다.', metadata={'source': '신한은행', 'row': 62, 'type': '인터넷뱅킹', 'timestamp': 1696755374.045472, 'id': '07146c5b-f74b-4eb0-b472-64f949ff96c7'}), Document(page_content="ask: 로그인은 어디서 하나요?\nInformation: 인터넷뱅킹에 로그인하시려면 신한은행 홈페이지에서 개인 → 뱅킹로그인 버튼을 클릭하시면 됩니다. 참고로 인터넷뱅킹에 로그인하시려면 우선 공동인증서 또는 금융인증서를 발급받으셔야 하며, '인증센터'에서 발급받으실 수 있습니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.", 

In [21]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

answer = chain.run(
    input_documents=search_keyword_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

ask: 즉시/예약 이체 서비스가 안되는 경우 어떻게 해야해?
Information: 이체서비스가 되지 않는 경우 여러가지 사유가 있을수 있으니, 신한은행 고객상담센터 1599-8000번으로 문의 주시면 자세히 상담해드리겠습니다.

ask: 로그인은 어디서 하나요?
Information: 인터넷뱅킹에 로그인하시려면 신한은행 홈페이지에서 개인 → 뱅킹로그인 버튼을 클릭하시면 됩니다. 참고로 인터넷뱅킹에 로그인하시려면 우선 공동인증서 또는 금융인증서를 발급받으셔야 하며, '인증센터'에서 발급받으실 수 있습니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 인터넷뱅킹을 하고자 하는데 인증서는 어디서 발급 하나요?
Information: 인증서는 신한은행에 인터넷뱅킹서비스가 가입되어 있으시면 신한은행 홈페이지 또는 신한 쏠(SOL)의 인증센터에서 발급 가능합니다. ※ 개인은 주민번호로 금융기관 통합하여 발급기관별, 용도별로 한 개의 인증서만 발급이 가능합니다. 예) 타은행에서 은행/신용카드/보험용(결제원) 인증서를 발급받은 경우 → 당행 은행/신용카드/보험용(결제원) 인증서 발급 불가

Question: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?



Assistant:

> Finished chain.

> Finished chain.
########################

### 키워드 검색 결과 (search_keyword_result)
bm25 score는 max_value로 normalization 되어 있음 (score range 0 - 1)

In [22]:
search_keyword_result

[Document(page_content='ask: 즉시/예약 이체 서비스가 안되는 경우 어떻게 해야해?\nInformation: 이체서비스가 되지 않는 경우 여러가지 사유가 있을수 있으니, 신한은행 고객상담센터 1599-8000번으로 문의 주시면 자세히 상담해드리겠습니다.', metadata={'source': '신한은행', 'row': 62, 'type': '인터넷뱅킹', 'timestamp': 1696755374.045472, 'id': '07146c5b-f74b-4eb0-b472-64f949ff96c7'}),
 Document(page_content="ask: 로그인은 어디서 하나요?\nInformation: 인터넷뱅킹에 로그인하시려면 신한은행 홈페이지에서 개인 → 뱅킹로그인 버튼을 클릭하시면 됩니다. 참고로 인터넷뱅킹에 로그인하시려면 우선 공동인증서 또는 금융인증서를 발급받으셔야 하며, '인증센터'에서 발급받으실 수 있습니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.", metadata={'source': '신한은행', 'row': 57, 'type': '인터넷뱅킹', 'timestamp': 1696755374.045458, 'id': 'bf8fef3b-836a-491c-8ee5-3fe67420a13a'}),
 Document(page_content='ask: 인터넷뱅킹을 하고자 하는데 인증서는 어디서 발급 하나요?\nInformation: 인증서는 신한은행에 인터넷뱅킹서비스가 가입되어 있으시면 신한은행 홈페이지 또는 신한 쏠(SOL)의 인증센터에서 발급 가능합니다. ※ 개인은 주민번호로 금융기관 통합하여 발급기관별, 용도별로 한 개의 인증서만 발급이 가능합니다. 예) 타은행에서 은행/신용카드/보험용(결제원) 인증서를 발급받은 경우 → 당행 은행/신용카드/보험용(결제원) 인증서 발급 불가', metadata={'source': '신한은행', 'row': 82, 'type': '인터넷뱅킹', 'timestam

## (3) OpenSearch Hybrid 검색

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) 위의 두 개의 결과의 스코어를 소팅해서, Top K 를 구함.
    - 단 (1), (2) 의 동일한 항목일 경우는 (1) 스코어 + (2) 스코어 = 종합 스코어로 적용함. 
    - 즉 동일한 항목이 있으면 (1), (2)를 합산하기에 스코어가 더 높을 가능성이 많음.
    - 종합 스코어는 최대 2.0 이 될 수 있음. 이유는 (1) 에서 최대 스코어, (2)에서 최대 스코어 일 경우에 1.0 + 1.0 = 2.0 이 됨

스코어 표준화 예시

<pre>
Docs    Score	    Normalized-Score

Doc1: 	0.0083		1.0

Doc2: 	0.0074		0.8900

Doc3: 	0.0071		0.8585

Example: 0.0074 / 0.0083 = 0.8900
</pre>

In [23]:
from utils.rag import interpolate_results, sort_score_search_results
from utils.rag import get_semantic_similar_docs, get_keyword_similar_docs, ensemble_results#, get_similiar_docs

In [24]:
# def search_hybrid(query, vectro_db, aws_client, is_filter, boolean_filter, k=10):
#     similar_docs_semantic = get_similiar_docs(
#         query,
#         vectro_db,
#         is_filter=True,
#         boolean_filter=boolean_filter,
#         weight_decay_rate=0.1,  # 가중치 감소 비올: 에: 0, 0.1, 0.2  ..쵀대 0.9 (값이 작을수록 semantic 검색결과 많이 반영)
#         k=k
#     )            

#     # print("##########################")
#     # print("similar_docs_semantic: \n", similar_docs_semantic)
#     # print("##########################")    
#     similar_docs_keyword = get_similiar_docs_with_keywords(
#         query,
#         aws_client,
#         index_name=index_name,
#         weight_decay_rate=0,
#         is_filter=True,
#         filter01=filter01,
#         filter02=filter02,
#         k=k
#     )
#     # print("##########################")    
#     # print("similar_docs_keyword: \n", similar_docs_keyword)    
#     # print("##########################")    
#     similar_docs = interpolate_results(similar_docs_semantic, similar_docs_keyword, k=k)
#     # print("##########################")    
#     # print("similar_docs: \n", similar_docs)
#     # print("##########################")    
#     # print("##--> Hybrid Search ")        
    
#     answer = chain.run(input_documents=similar_docs, question=query)
    
#     return answer

In [28]:
def search_hybrid(**kwargs):
    
    assert "query" in kwargs, "Check your query"
    assert "vector_db" in kwargs, "Check your vector_db"
    assert "index_name" in kwargs, "Check your index_name"
    assert "os_client" in kwargs, "Check your os_client"
    assert "qa_chain" in kwargs, "Check your qa_chain"

    similar_docs_semantic = get_semantic_similar_docs(
        vector_db=kwargs["vector_db"],
        query=kwargs["query"],
        k=kwargs.get("k", 5),
        hybrid=True
    )

    similar_docs_keyword = get_keyword_similar_docs(
        query=query,
        minimum_should_match=kwargs.get("minimum_should_match", 0),
        filter=kwargs.get("filter", []),
        index_name=kwargs["index_name"],
        os_client=kwargs["os_client"],
        k=kwargs.get("k", 5),
        hybrid=True
    )

    similar_docs_ensemble = ensemble_results(
        doc_lists = [similar_docs_semantic, similar_docs_keyword],
        weights = kwargs.get("ensemble_weights", [.5, .5]),
        algorithm="simple_weighted",
        c=60,
        k=kwargs.get("k", 5)
    )
    similar_docs_ensemble = list(map(lambda x:x[0], similar_docs_ensemble))

    answer = kwargs["qa_chain"].run(
        input_documents=similar_docs_ensemble,
        question=query
    )

    return answer


In [31]:
%%time

filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
#query = "타기관OTP 등록 방법 알려주세요"
#query = "웹싸이트 계정몇 개 사용할 수 있나요?"

answer = search_hybrid(
    query=query,
    vector_db=vector_db,
    k=5,
    index_name=index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    ensemble_weights=[.3, .5],
    qa_chain=chain
)

keyword search query: 
{'query': {'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}},
                               {'term': {'metadata.source': '신한은행'}}],
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '홈페이지 이용자아이디 여러 개 '
                                                          '사용할 수 있나요?'}}}]}},
 'size': 5}


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

ask: 즉시/예약 이체 서비스가 안되는 경우 어떻게 해야해?
Information: 이체서비스가 되지 않는 경우 여러가지 사유가 있을수 있으니, 신한은행 고객상담센터 1599-8000번으로 문의 주시면 자세히 상담해드리겠습니다.

ask: 로그인은 어디서 하나요?
Information: 인터넷뱅킹에 로그인하시려면 신한은행 홈페이지에서 개인 → 뱅킹로그인 버튼을 클릭하시면 됩니다. 참고로 인터넷

In [32]:
print (f'question: {query}')
print (f'response: {answer}')

question: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
response:  정보에 따르면 신한은행 인터넷뱅킹 이용자 아이디는 주민등록번호를 기준으로 1개만 발급됩니다. 따라서 홈페이지 이용자 아이디를 여러 개 사용할 수는 없습니다. 1개의 주민등록번호에 한 개의 이용자 아이디만 발급이 가능합니다.
